<a href="https://colab.research.google.com/github/arjasc5231/moodots/blob/main/SER/RNN/simpleRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
import os
import sklearn as sk
import librosa

In [ ]:
learning_rate = 0.001
training_epochs = 61
batch_size = 100

In [ ]:
X_train, X_test, Y_train, Y_test = np.load("/content/drive/MyDrive/team_runner/colab/dataset/emoDB/forRNN_downsamp.npy", allow_pickle=True)

print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(1585, 5000)
(1585,)
(529, 5000)
(529,)


In [ ]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
Y_train = to_categorical(Y_train, 7)
Y_test = to_categorical(Y_test, 7)

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train)).shuffle(buffer_size=100000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test)).batch(batch_size)

In [ ]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(1585, 5000, 1)
(1585, 7)
(529, 5000, 1)
(529, 7)


In [ ]:
model = keras.Sequential()
model.add(keras.layers.SimpleRNN(100, input_shape=(5000, 1), activation=tf.nn.relu))
model.add(keras.layers.Dense(7))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 100)               10200     
_________________________________________________________________
dense_1 (Dense)              (None, 7)                 707       
Total params: 10,907
Trainable params: 10,907
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(
        y_pred=logits, y_true=labels, from_logits=True))    
    return loss

In [ ]:
def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

In [ ]:
def evaluate(model, images, labels):
    logits = model(images, training=False)
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return accuracy

In [ ]:
optimizer = tf.optimizers.Adam(learning_rate=learning_rate)

In [ ]:
print('Learning started. It takes sometime.')

for epoch in range(training_epochs):
  avg_loss = 0.
  avg_train_acc = 0.
  avg_test_acc = 0.
  train_step = 0
  test_step = 0

  for waves, labels in train_dataset:
    grads = grad(model, waves, labels)                
    optimizer.apply_gradients(zip(grads, model.variables))
    loss = loss_fn(model, waves, labels)
    acc = evaluate(model, waves, labels)
    avg_loss = avg_loss + loss
    avg_train_acc = avg_train_acc + acc
    train_step += 1
  avg_loss = avg_loss / train_step
  avg_train_acc = avg_train_acc / train_step
  
  for waves, labels in test_dataset:        
    acc = evaluate(model, waves, labels)        
    avg_test_acc = avg_test_acc + acc
    test_step += 1
  avg_test_acc = avg_test_acc / test_step   

  print('Epoch:', '{}'.format(epoch + 1), 'loss =', '{:.8f}'.format(avg_loss), 
        'train accuracy = ', '{:.4f}'.format(avg_train_acc), 
        'test accuracy = ', '{:.4f}'.format(avg_test_acc))

Learning started. It takes sometime.
Epoch: 1 loss = 1.93055236 train accuracy =  0.2056 test accuracy =  0.2245
Epoch: 2 loss = 1.91025805 train accuracy =  0.2241 test accuracy =  0.2261
Epoch: 3 loss = 1.91911495 train accuracy =  0.2332 test accuracy =  0.2328
Epoch: 4 loss = 1.91337740 train accuracy =  0.2125 test accuracy =  0.2419
Epoch: 5 loss = 1.90747023 train accuracy =  0.2465 test accuracy =  0.2261
Epoch: 6 loss = 1.90178657 train accuracy =  0.2256 test accuracy =  0.2302
Epoch: 7 loss = 1.88682854 train accuracy =  0.2479 test accuracy =  0.2886
Epoch: 8 loss = 1.91168141 train accuracy =  0.2149 test accuracy =  0.2302
Epoch: 9 loss = 1.89790797 train accuracy =  0.2646 test accuracy =  0.2876
Epoch: 10 loss = 1.89139915 train accuracy =  0.2861 test accuracy =  0.2967
Epoch: 11 loss = 1.87186384 train accuracy =  0.2832 test accuracy =  0.2951
Epoch: 12 loss = 1.85485935 train accuracy =  0.2801 test accuracy =  0.2826
Epoch: 13 loss = 1.84477937 train accuracy =  0.

In [ ]:
conf_mat = [[0]*7 for i in range(7)] #mat[real_label]=predicted_label list

for images, labels in test_dataset:
  logits = model(images, training=False)
  logits_max = tf.math.argmax(logits,1)
  labels_max = tf.math.argmax(labels,1)
  for i in range(len(logits)): conf_mat[labels_max[i]][logits_max[i]]+=1

for i in range(7): print(conf_mat[i])

[33, 0, 3, 3, 9, 0, 0]
[0, 19, 4, 0, 0, 12, 13]
[0, 1, 4, 0, 0, 0, 0]
[3, 3, 6, 8, 1, 2, 2]
[5, 0, 1, 5, 10, 0, 2]
[0, 2, 0, 1, 1, 19, 0]
[0, 2, 1, 2, 0, 0, 5]


In [ ]:
conf_mat_normal = []
for i in range(7):
  s = sum(conf_mat[i])
  conf_mat_normal.append(list(map(lambda x:(x/s)*100, conf_mat[i])))

label = ['anger','boredom','disgust','fear','happy','sad','neutral']
print('\t'+'\t'.join(label))
for i in range(7):
  print(label[i], end='')
  for j in range(7): print('\t%2.0f'%conf_mat_normal[i][j], end=' ')
  print()

	anger	boredom	disgust	fear	happy	sad	neutral
anger	69 	 0 	 6 	 6 	19 	 0 	 0 
boredom	 0 	40 	 8 	 0 	 0 	25 	27 
disgust	 0 	20 	80 	 0 	 0 	 0 	 0 
fear	12 	12 	24 	32 	 4 	 8 	 8 
happy	22 	 0 	 4 	22 	43 	 0 	 9 
sad	 0 	 9 	 0 	 4 	 4 	83 	 0 
neutral	 0 	20 	10 	20 	 0 	 0 	50 
